In [ ]:
import jax
from jax import random
import jax.numpy as jnp

In [ ]:
# some coefficients to play with

I = 2 # number of inputs
O = 3 # number of outputs
na = 4
nb = 5

In [ ]:
key = random.key(200)
a_coeff = random.normal(key, (O, I, na)) 
b_coeff = random.normal(key, (O, I, nb)) 

In [ ]:
# initial conditions
u_carry = jnp.array([O, I, 0.0, 0.0]) # u-1, u-2
y_carry = jnp.array([O, I, 0.0, 0.0]) # y-1, y-2

In [ ]:
# simulate one step
u_step = jnp.array(I, 1.0) # u0

u_carry = jnp.r_[u_step, u_carry]
y_new = jnp.dot(b_coeff, u_carry) - jnp.dot(a_coeff, y_carry)
u_carry = u_carry[:-1]
y_carry = jnp.r_[y_new, y_carry][:-1]


y_new

In [ ]:
# simulate one step with a function (for later use with scan)
def filter_step(carry, u_step):

    u_carry, y_carry = carry
    u_carry = jnp.r_[u_step, u_carry]
    y_new = jnp.dot(b_coeff, u_carry) - jnp.dot(a_coeff, y_carry)

    u_carry = u_carry[:-1]
    y_carry = jnp.r_[y_new,  y_carry][:-1]
    carry = (u_carry, y_carry)
    return carry, y_new

new_carry, new_y = filter_step((u_carry, y_carry), ut)
y_carry

In [ ]:
# some random data
key = random.key(200)
u = jax.random.normal(key, (200,))

In [ ]:
# simulate with scan
u_carry = jnp.array([0.0, 0.0]) # u-1, u-2
y_carry = jnp.array([0.0, 0.0]) # y-1, y-2
carry = (u_carry, y_carry)
last_carry, y = jax.lax.scan(filter_step, carry, u)

In [ ]:
# simulate with scipy
import scipy.signal as signal
y_lfilter = signal.lfilter(b_coeff, jnp.r_[1.0, a_coeff], u, axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(y, "k")
plt.plot(y_lfilter, "b")
plt.plot(y-y_lfilter, "r");